In [ ]:
!pip install lightgbm catboost

   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
    --------------------------------------- 1.6/102.4 MB 13.8 MB/s eta 0:00:08
   -- ------------------------------------- 7.6/102.4 MB 24.6 MB/s eta 0:00:04
   ----- ---------------------------------- 13.9/102.4 MB 26.6 MB/s eta 0:00:04
   ------- -------------------------------- 19.4/102.4 MB 26.4 MB/s eta 0:00:04
   ---------- ----------------------------- 26.2/102.4 MB 27.7 MB/s eta 0:00:03
   ------------- -------------------------- 33.3/102.4 MB 28.7 MB/s eta 0:00:03
   --------------- ------------------------ 40.9/102.4 MB 30.0 MB/s eta 0:00:03
   ------------------ --------------------- 46.9/102.4 MB 30.0 MB/s eta 0:00:02
   -------------------- ------------------- 53.0/102.4 MB 29.9 MB/s eta 0:00:02
   ------------------------ --------------- 61.6/102.4 MB 30.9 MB/s eta 0:00:02
   --------------------------- ------------ 70.0/102.4 MB 31.7 MB/s eta 0:00:02
   ------------------------------ --------- 77.3/10

In [3]:
import json
import numpy as np
import pandas as pd
import joblib
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# ================================
# Caminhos
# ================================
json_path = r"C:\Users\Willgnner\Documents\TJ-GO\Perfil-Conciliação-Xgboost-GCP-V2\dados\cejusc_14052025\dados_processos_cejusc_14052025_V1_enriquecidos.json"
embedding_path = r"C:\Users\Willgnner\Documents\TJ-GO\Perfil-Conciliação-Xgboost-GCP-V2\dados\Embeddings\dados_processos_cejusc_14052025_V1_enriquecidos_Embeddings.npy"
modelo_saida = r"C:\Users\Willgnner\Documents\TJ-GO\Perfil-Conciliação-Xgboost-GCP-V2\Notebooks\LightGBM\modelo_lightgbm_v1.pkl"

# ================================
# Carga dos dados
# ================================
with open(json_path, 'r', encoding='utf-8') as f:
    dados = json.load(f)

df = pd.DataFrame(dados)
embeddings = np.load(embedding_path)
df = df.iloc[:embeddings.shape[0]]

In [4]:
# ================================
# Engenharia de features
# ================================
prefixos = ("tem_", "n_", "tipo_", "cita_", "pede_", "houve_", "tentou_", "usa_", "problema_", "citou_")
colunas_struct = [c for c in df.columns if c.startswith(prefixos)]
cat_cols = [c for c in colunas_struct if df[c].dtype == 'object']
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
colunas_final = [c for c in df.columns if c.startswith(prefixos)]
X_struct = df[colunas_final].fillna(0).astype(float).values

X = np.hstack((embeddings, X_struct))
y = df["classificacao"].map({'frutifero': 1, 'infrutifero': 0}).values

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# ================================
# Treinar modelo
# ================================
modelo = LGBMClassifier(
    n_estimators=600,
    learning_rate=0.03,
    max_depth=5,
    class_weight='balanced',
    subsample=0.7,
    colsample_bytree=0.5,
    random_state=42
)
modelo.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 3855, number of negative: 4987
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.197303 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 783672
[LightGBM] [Info] Number of data points in the train set: 8842, number of used features: 3092
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

LGBMClassifier(class_weight='balanced', colsample_bytree=0.5,
               learning_rate=0.03, max_depth=5, n_estimators=600,
               random_state=42, subsample=0.7)

In [5]:
# ================================
# Melhor threshold
# ================================
y_proba = modelo.predict_proba(X_test)[:, 1]
thresholds = np.linspace(0.1, 0.9, 30)
f1_scores = [f1_score(y_test, (y_proba > t).astype(int)) for t in thresholds]
melhor_threshold = thresholds[np.argmax(f1_scores)]
y_pred = (y_proba > melhor_threshold).astype(int)

c:\Users\Willgnner\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [6]:
# ================================
# Avaliação
# ================================
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
total_erros = (y_test != y_pred).sum()

print("\nAvaliação - LightGBM")
print(f"  - Acurácia:      {acc:.4f}")
print(f"  - Precisão:      {prec:.4f}")
print(f"  - Recall:        {rec:.4f}")
print(f"  - F1-score:      {f1:.4f}")
print(f"  - Erros:         {total_erros} / {len(y_test)}")
print(f"  - Threshold ótimo: {melhor_threshold:.2f}")

joblib.dump(modelo, modelo_saida)


Avaliação - LightGBM
  - Acurácia:      0.5242
  - Precisão:      0.4749
  - Recall:        0.8641
  - F1-score:      0.6130
  - Erros:         1052 / 2211
  - Threshold ótimo: 0.32


['C:\\Users\\Willgnner\\Documents\\TJ-GO\\Perfil-Conciliação-Xgboost-GCP-V2\\Notebooks\\LightGBM\\modelo_lightgbm_v1.pkl']

In [ ]:
import json
import numpy as np
import pandas as pd
import joblib

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix
)

# -----------------------------
# CAMINHOS
# -----------------------------
caminho_json_novo = r"C:\Users\Willgnner\Documents\TJ-GO\Perfil-Conciliação-Xgboost-GCP-V2\dados\cejusc_03072025\dados_processos_cejusc_03072025_V1_enriquecidos.json"
caminho_embeddings_novo = r"C:\Users\Willgnner\Documents\TJ-GO\Perfil-Conciliação-Xgboost-GCP-V2\dados\Embeddings\dados_processos_cejusc_03072025_V1_enriquecidos_Embeddings.npy"
caminho_modelo = r"C:\Users\Willgnner\Documents\TJ-GO\Perfil-Conciliação-Xgboost-GCP-V2\Notebooks\LightGBM\modelo_lightgbm_v1.pkl"  # Ou o XGBoost, se preferir

# -----------------------------
# CARGA DOS DADOS
# -----------------------------
with open(caminho_json_novo, 'r', encoding='utf-8') as f:
    dados = json.load(f)

df_novo = pd.DataFrame(dados)
vetores_novos = np.load(caminho_embeddings_novo)
df_novo = df_novo.iloc[:vetores_novos.shape[0]]

# -----------------------------
# FEATURE ENGINEERING
# -----------------------------
prefixos = ("tem_", "n_", "tipo_", "cita_", "pede_", "houve_", "tentou_", "usa_", "problema_", "citou_")
colunas_struct = [col for col in df_novo.columns if col.startswith(prefixos)]
cat_cols = [col for col in colunas_struct if df_novo[col].dtype == 'object']
df_novo = pd.get_dummies(df_novo, columns=cat_cols, drop_first=True)
colunas_struct_final = [col for col in df_novo.columns if col.startswith(prefixos)]
X_struct_novo = df_novo[colunas_struct_final].fillna(0).astype(float).values
X_novo = np.hstack((vetores_novos, X_struct_novo))

# -----------------------------
# INFERÊNCIA
# -----------------------------
modelo = joblib.load(caminho_modelo)
y_pred = modelo.predict(X_novo)

# -----------------------------
# AVALIAÇÃO (se rótulo real existir!)
# -----------------------------
if "classificacao" in df_novo.columns:
    y_true = df_novo["classificacao"].map({'frutifero': 1, 'infrutifero': 0}).values

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    total_erros = (y_true != y_pred).sum()

    print(f"Avaliação na nova base:")
    print(f"  - Acurácia:      {acc:.4f}")
    print(f"  - Precisão:      {prec:.4f}")
    print(f"  - Recall:        {rec:.4f}")
    print(f"  - F1-score:      {f1:.4f}")
    print(f"  - Total de erros: {total_erros} / {len(y_true)}")

else:
    print("A coluna 'classificacao' não existe nos dados! Não é possível calcular métricas.")

Avaliação na nova base:
  - Acurácia:      0.5546
  - Precisão:      0.5091
  - Recall:        0.4915
  - F1-score:      0.5002
  - Total de erros: 1509 / 3388


c:\Users\Willgnner\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
